In [1]:
def fib(n):
    """nを0以上の整数とする
       n番目のフィボナッチ数を返す"""
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [2]:
def fastFib(n, memo = {}):
    """nを0以上の整数とする． memoは再帰呼び出しによってのみ使われる
       n番目のフィボナッチ数を返す"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [3]:
%time fastFib(120)

CPU times: user 704 µs, sys: 280 µs, total: 984 µs
Wall time: 852 µs


8670007398507948658051921L

In [4]:
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = float(v)
        self.weight = float(w)
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        result = '<' + self.name + ', ' + str(self.value)\
                 + ', ' + str(self.weight) + '>'
        return result

In [5]:
def maxVal(toConsider, avail):
    """toConsiderを品物のリスト, availを重さとする
       それらをパラメータとする0/1ナップサック問題の解である
       総重量と品物のリストからなるタプルを返す"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        #右側の分岐のみを探索する
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        #左側の分岐を探索する
        withVal, withToTake = maxVal(toConsider[1:],
                                     avail - nextItem.getWeight())
        withVal += nextItem.getValue()
        #右側の分岐を探索する
        withoutVal, withoutToTake = maxVal(toConsider[1:],
                                           avail)
        #よりよい分岐を選択
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

In [6]:
def smallTest():
    names = ['a', 'b', 'c', 'd']
    vals = [6, 7, 8, 9]
    weights = [3, 3, 2, 5]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i], vals[i], weights[i]))
    val, taken = maxVal(Items, 5)
    for item in taken:
        print item
    print '選択した項目の合計金額 =', val

In [7]:
smallTest()

<c, 8.0, 2.0>
<b, 7.0, 3.0>
選択した項目の合計金額 = 15.0


In [8]:
import random

In [9]:
def buildManyItems(numItems, maxVal, maxWeight):
    items = []
    for i in range(numItems):
        items.append(Item(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxWeight)))
    return items

In [10]:
def bigTest(numItems):
    items = buildManyItems(numItems, 10, 10)
    val, taken = maxVal(items, 40)
    print '選択した項目'
    for item in taken:
        print item
    print '選択した項目の合計金額 =', val

In [11]:
bigTest(10)

選択した項目
<9, 7.0, 5.0>
<7, 7.0, 5.0>
<6, 7.0, 4.0>
<5, 10.0, 1.0>
<3, 8.0, 3.0>
<2, 6.0, 9.0>
<1, 10.0, 3.0>
<0, 8.0, 8.0>
選択した項目の合計金額 = 63.0


In [12]:
def fastMaxVal(toConsider, avail, memo = {}):
    """toConsiderを品物のリスト, availを重さ，
       memoは再帰呼び出しによってのみ使われるとする
       それらをパラメータとする0/1ナップサック問題の解である
       総重量と品物のリストからなるタプルを返す"""
    if (len(toConsider), avail) in memo:
        result = memo[(len(toConsider), avail)]
    elif toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        #Explore right branch only
        result = fastMaxVal(toConsider[1:], avail, memo)
    else:
        nextItem = toConsider[0]
        #Explore left branch
        withVal, withToTake =\
                 fastMaxVal(toConsider[1:],
                            avail - nextItem.getWeight(), memo)
        withVal += nextItem.getValue()
        #Explore right branch
        withoutVal, withoutToTake = fastMaxVal(toConsider[1:],
                                                avail, memo)
        #Choose better branch
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    memo[(len(toConsider), avail)] = result
    return result

In [13]:
def bigTest(numItems):
    items = buildManyItems(numItems, 10, 10)
    val, taken = maxVal(items, 1000)
    print '選択した項目'
    for item in taken:
        print item
    print '選択した項目の合計金額 =', val

In [14]:
def buildManyItems(numItems, maxVal, maxWeight):
    items = []
    for i in range(numItems):
        items.append(Item(str(i),
                          random.randint(1, maxVal),
                          random.randint(1, maxWeight) * random.random()))
    return items